In [1]:
# import utility modules
import pandas as pd
import numpy as np
import configparser
import os

import numpy as np
import pandas as pd
from scipy.optimize import minimize
from helpers.helper_classes import AddFeatureNames, Gene_SPCA
from joblib import dump, load


In [2]:
np.random.seed(2023)


In [3]:
# Read config.ini file
config = configparser.ConfigParser()
config.read('config.ini')

os.chdir(config['PATH']['ROOT_DIR'])

# Read data
data = load(config['PATH']['DATA_DIR'] + '/microarray-data-dict.lib')

# Read parameters
SEED = config.getint('PARAMS', 'SEED')
N_COMPONENTS = config.getint('PARAMS', 'N_COMPONENTS')

In [5]:
X = data['golub']['none']['X_train']

from sklearn.base import BaseEstimator, TransformerMixin
from tqdm import tqdm

In [6]:
spca = Gene_SPCA()
spca.fit(X, verbose = 1)

Progress based on max iterations:


100%|██████████| 200/200 [00:59<00:00,  3.37it/s]


Gene_SPCA()

In [182]:
X2.shape

(48, 20)

In [152]:
# Pre step 1: get and center data

X = np.random.random((100,2000))
mean_subtr = np.mean(X, axis = 0)

# Important, center data
X = X - mean_subtr

Algorithm 1. General SPCA algorithm
1. let A start at V[,1:k], the loadings of the first k ordinary principal components
2. given A = [$\alpha_1, \cdots, \alpha_k$] solve


In [280]:
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm import tqdm

class Gene_SPCA(BaseEstimator, TransformerMixin):
    def __init__(self, n_comps = 20, max_iter = 200, tol = 0.001, improve_tol = 0.00001, l1 = 5):
        self.max_iter = max_iter
        self.tol = tol
        self.improve_tol = improve_tol
        self.n_comps = n_comps
        self.l1 = l1
        self.loadings = None
        self.hasFit = False

    def fit(self, X, y = None, verbose = 0):

        if verbose: 
            print("Progress based on max iterations:")
            pbar = tqdm(total = self.max_iter)

        # Step 1: Setup first iteration
        U, _, Vt = np.linalg.svd(X, full_matrices=False)
        A = Vt.T[:,:self.n_comps]
        B = np.zeros((A[:,0].shape[0], self.n_comps))
        XtX = X.T @ X
        iter = 0
        diff, diff_improve = 100, 100

        # Loop of step 2 and 3 until convergence / maxiter:
        while iter < self.max_iter and diff > self.tol and diff_improve > self.improve_tol:
            B_old = np.copy(B)
            
            # Update B (step 2*)
            input = A.T @ XtX
            for i in range(self.n_comps):
                B[:, i] = self._soft_threshold(input[i,:], 10)
            
            # Monitor change
            diff_old = diff
            diff = self._max_diff(B_old, B)
            diff_improve = np.abs(diff - diff_old)

            # Update A (step 3)
            A_old = A    
            Un, s, Vnt = np.linalg.svd(XtX @ B, full_matrices=False)
            A = Un @ Vnt

            if verbose: pbar.update(1)
            iter = iter + 1
        pbar.close()

        # Normalize loadings after loop
        B = self._normalize_mat(B)
        self.loadings = B
        return self
    
    def transform(self, X, y = None):
        return X @ self.loadings
    
    def _soft_threshold(self, vec, l1):
        temp = np.maximum(0, (np.abs(vec) - l1 / 2))
        return temp * np.sign(vec)

    def _max_diff(self, X1, X2):
        return np.max(np.abs(X1 - X2))

    def _normalize_mat(self, X):
        for i in range(X.shape[1]):
            X[:,i] = X[:,i] / np.maximum(np.linalg.norm(X[:,i]), 1)
        return X   


In [281]:
transform = Gene_SPCA()
transform.fit(X, verbose = 1)
traintest = transform.transform(data['golub']['none']['X_train'])

Progress based on max iterations:


 44%|████▎     | 87/200 [00:29<00:38,  2.94it/s]


In [172]:
# Step 1: Get first k loadings of ordinary principal components
U, s, Vt = np.linalg.svd(X, full_matrices=False)
V = Vt.T
D = np.zeros((U.shape[0], Vt.shape[0]))
D[:s.size, :s.size] = np.diag(s)



In [194]:
# Step 2: solve elastic net problem for j in 1 ... k

def objective_elastic_net(beta, alpha, X, l, l2):
    assert len(beta) == len(alpha), "shape of principal comp. vector alpha does not coincide with beta"
    return (alpha - beta).T @ X.T @ X @ (alpha - beta) + l * np.linalg.norm(beta) ** 2 + l2 * np.linalg.norm(beta)

def soft_threshold(vec, l1):
    temp = np.maximum(0, (np.abs(vec) - l1 / 2))
    return temp * np.sign(vec)

def max_diff(X1, X2):
    return np.max(np.abs(X1 - X2))

def normalize_mat(X):
    for i in range(X.shape[1]):
        X[:,i] = X[:,i] / np.linalg.norm(X[:,i])
    return X

# For normal SPCA
# B[:,i] = minimize(objective_elastic_net, np.zeros(B[:,i].shape[0]), args = (A[:,i], X, 1, 2)).x


In [238]:
from scipy.linalg import sqrtm


k = 10
A = V[:,:k]
B = np.zeros((A[:,0].shape[0], k))
XtX = X.T @ X

tol = 0.001
maxiter = 200
iter = 0
diff = 10

while iter < maxiter and diff > tol:

    B_old = np.copy(B)

    input = A.T @ XtX
    for i in range(k):
        x = input[i,:]
        B[:, i] = soft_threshold(x, 10)
    
    diff = max_diff(B_old, B)

    # Update A (step 3)
    A_old = A    
    Un, s, Vnt = np.linalg.svd(XtX @ B, full_matrices=False)
    A = Un @ Vnt

    print(iter)
    print(diff)    
    print(np.count_nonzero(B))

    iter = iter + 1

B = normalize_spca(B)




0
1513.1375378889386
69450
1
1.7448700991683381
69441
2
1.3336661661349325
69430
3
1.3243365202627047
69440
4
1.3147965143171092
69443
5
1.306580337606647
69442
6
1.297287141508832
69458
7
1.2889679095624587
69462
8
1.282342662367256
69460
9
1.2761621587582255
69463
10
1.2688781972920253
69468
11
1.261676639080207
69458
12
1.254208358805073
69459
13
1.2465486852736518
69453
14
1.2397049254986854
69434
15
1.232822754453096
69438
16
1.2260402711665819
69433
17
1.2186481138058411
69423
18
1.2116439775413568
69426
19
1.204619939615526
69424
20
1.1979500276377735
69424
21
1.192465491457554
69417
22
1.187098650570661
69406
23
1.1811105906828914
69401
24
1.1739720547341363
69392
25
1.1658345481362034
69390
26
1.156860604379915
69390
27
1.1475184051748073
69384
28
1.138345763232735
69377
29
1.1300102225294353
69378
30
1.1224313496653977
69385
31
1.1153783129547747
69377
32
1.1089541093639923
69383
33
1.1026896392392516
69378
34
1.0964504133966528
69373
35
1.091261696378588
69370
36
1.085751246

In [277]:
from sklearn.decomposition import PCA, SparsePCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, roc_auc_score, roc_curve, RocCurveDisplay, f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


newX = X @ B
testX = data['golub']['none']['X_test'] @ B
y_train =  data['golub']['none']['y_train']
y_test =  data['golub']['none']['y_test']

clf = LogisticRegression(random_state= 2023)
clf.fit(traintest, data['golub']['none']['y_train'])
clf.score(transform.transform(data['golub']['none']['X_test']), data['golub']['none']['y_test'])


c:\Users\caspa\Desktop\VU_git\vu-case-study-eds\.venv\Lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9583333333333334

In [275]:
traintest.shape

(24, 20)

In [243]:
X.shape

(48, 7129)